In [25]:
import os
import pymorphy2
import numpy as np
from nltk.corpus import stopwords
from string import punctuation

In [21]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [51]:
import pandas as pd

In [15]:
import re

In [11]:
stop = stopwords.words('russian')

In [28]:
morph = pymorphy2.MorphAnalyzer()

2017-03-29 22:15:25,463 : INFO : Loading dictionaries from /usr/local/lib/python3.5/dist-packages/pymorphy2_dicts/data
2017-03-29 22:15:33,880 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [43]:
d = {}
izvest = []
tehmol = []
anekdots = []
for root, dirs, files in os.walk('hw6-texts'):
    for file in files:
        if '/teh_mol/' in os.path.join(root, file):
            tehmol.append(open(os.path.join(root, file)).read())
        elif '/izvest/' in os.path.join(root, file):
            izvest.append(open(os.path.join(root, file)).read())
        else:
            anekdots.append(open(os.path.join(root, file)).read())
        #print(file, d[file])

In [23]:
m = 'ruscorpora_1_300_10.bin.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.Word2Vec.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.Word2Vec.load(m)

2017-03-29 20:51:03,410 : INFO : loading projection weights from ruscorpora_1_300_10.bin.gz
2017-03-29 20:52:17,567 : INFO : loaded (184973, 300) matrix from ruscorpora_1_300_10.bin.gz


In [38]:
model.init_sims(replace=True)

2017-03-29 22:30:14,569 : INFO : precomputing L2-norms of word weight vectors


In [41]:
def fix_text(text):
    translation = str.maketrans("","", punctuation)
    text = text.lower().translate(translation)
    text = text.split()
    t = []
    for i in text:
        if i not in stop:
            p = morph.parse(i)[0]
            try:
                t.append(p.normal_form+'_'+p.tag.POS)
            except TypeError:
                pass
    fin = []
    for w in t:
        if w in model:
            fin.append(model[w])
    return(np.mean(fin, axis=0))
    
fix_text('''Завелся в лесу страшный зверек -- ебунчик. Совсем от него житья не стало, так он всех задолбал. И пошли звери с поклоном к Змею Горынычу.
-- Выручай, -- говорят, -- трехголовый! Спасу нет от этого паразита, как отвлечешься -- он тут как тут, и деваться некуда...
И пошел Змей разбираться с этим чудом в лес. Долго ли, коротко ли, вышел он на полянку, посреди полянки -- пенек, а на пеньке сидит кто-то.
-- Ты кто такой? -- спрашивает Змей.
-- Да вот, местные ебунчиком прозвали, -- отвечает зверек.
Тут Змей как затопает, крыльями захлопает, огнем пышет, только что дым из ушей не пускает -- страх наводит. Пошумел, пошумел и грозно так спрашивает:
-- Что, чучело, страшно?!
-- Ой, страшно! -- ответил зверек. -- В первый раз такого страшного е**ть буду!''')

array([-0.02321471,  0.00596449,  0.02933581, -0.01512913,  0.00654654,
       -0.00030561,  0.01774949,  0.02480726, -0.00544954, -0.02032459,
       -0.00751817, -0.01634403,  0.027398  ,  0.000829  ,  0.00706642,
       -0.01488593,  0.00108798,  0.01233504,  0.03739146,  0.00833952,
        0.04072807, -0.01425894,  0.01527308,  0.02892763, -0.0100497 ,
       -0.01975117,  0.00811037, -0.02558992, -0.0214788 , -0.03356385,
       -0.00093852,  0.01987472,  0.0641589 , -0.00742231, -0.0203022 ,
       -0.01246664,  0.01040726, -0.01347763,  0.01489009, -0.01997107,
        0.04748747,  0.00886253,  0.03080262, -0.02744259,  0.01133794,
        0.01501525,  0.02955781, -0.00924788,  0.05759448, -0.01838891,
        0.00307633,  0.00933181,  0.01048271, -0.00962422,  0.01636022,
       -0.01671817, -0.0254766 ,  0.01933217, -0.05067044, -0.00870224,
       -0.00912203,  0.01586975,  0.00792406, -0.03559281,  0.00709056,
       -0.00589136, -0.07270867,  0.04466733, -0.00644757,  0.02

преобразовываем тексты в вектора:

In [44]:
izvest = [fix_text(t) for t in izvest]
tehmol = [fix_text(t) for t in tehmol]
anekdots = [fix_text(t) for t in anekdots]

In [47]:
izvest = np.array(izvest)
tehmol = np.array(tehmol)
anekdots = np.array(anekdots)

In [48]:
len(izvest[0])

300

In [49]:
cols = [str(i) for i in range(1, 301)]

In [56]:
iz_pd = pd.DataFrame(izvest, columns=cols)
iz_pd['class'] = 0
an_pd = pd.DataFrame(anekdots, columns=cols)
an_pd['class'] = 1
tc_pd = pd.DataFrame(tehmol, columns=cols)
tc_pd['class'] = 2

In [61]:
alldata = pd.concat([iz_pd, an_pd, tc_pd])

In [67]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, make_scorer, precision_score, recall_score

In [69]:
clf = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=4)
print('f1-score: ', cross_val_score(clf, alldata.iloc[:, :-1], alldata['class'], scoring='f1_micro'))

f1-score:  [ 0.91269841  0.86507937  0.89430894]


f1-мера при кросс-валидации

In [71]:
clf.fit(alldata.iloc[:, :-1], alldata['class'])
print(classification_report(clf.predict(alldata.iloc[:, :-1]), alldata['class'], target_names=['izvest', 'anekdots', 'tecmol']))

             precision    recall  f1-score   support

     izvest       0.85      0.87      0.86       122
   anekdots       0.98      0.96      0.97       128
     tecmol       0.86      0.86      0.86       125

avg / total       0.90      0.90      0.90       375



оптимистичная оценка классификатора